In [ ]:
import geopandas as gpd
import osmnx as ox
import pandas as pd
from shapely import Point

ox.settings.use_cache = True
ox.settings.log_console = False

In [2]:
CITY = "budapest"
CRS = 23700
STOPS_FILENAME = "Budapest_stops_with_centrality.csv"

In [4]:
stops = pd.read_csv(f"../data/stops/{CITY}/{STOPS_FILENAME}", engine="pyarrow")
stops["geometry"] = stops.apply(lambda x: Point(x["stop_lon"], x["stop_lat"]), axis=1)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326)
stops.to_crs(CRS, inplace=True)
stops.head(3)

,Node,Eigenvector Centrality,Degree Centrality,Closeness Centrality,Betweenness Centrality,stop_id,clust,stop_lat,stop_lon,geometry
0,1.0,3.632075e-15,0.000437,0.033463,0.00000,044369,1,47.175605,18.941796,POINT (641991.147 203505.079)
1,1.0,3.632075e-15,0.000437,0.033463,0.00000,088453,1,47.174793,18.941817,POINT (641992.615 203414.809)
2,2.0,1.878265e-14,0.001311,0.034621,0.00262,CSF04632,2,47.227924,18.946067,POINT (642322.523 209320.816)


In [ ]:
boundary = gpd.read_file(f"../data/geojson/{CITY}/boundary.geojson").set_crs(4326)
boundary.to_crs(CRS, inplace=True)

In [10]:
boundary.centroid

0    POINT (654535.962 237790.095)
dtype: geometry